## Acquire Data

In [1]:
import os
import pickle

import numpy as np

In [2]:
os.chdir('/content/drive/MyDrive/stock_charts_images')

In [3]:
list_of_stock_chart_pkl_files = os.listdir()

In [4]:
%%time

with open(list_of_stock_chart_pkl_files[1], 'rb') as f:
    data = pickle.load(f)

CPU times: user 3.05 ms, sys: 558 ms, total: 561 ms
Wall time: 7.17 s


In [5]:
type(data), len(data)

(tuple, 2)

In [6]:
type(data[0]), data[0].shape

(numpy.ndarray, (2172, 299, 299, 3))

In [7]:
type(data[1]), data[1].shape

(numpy.ndarray, (2172,))

In [8]:
np.unique(data[1])

array([-0.58323352, -0.56096147, -0.34562271, ...,  0.35772196,
        0.50475306,  0.81008261])

In [9]:
X, y = data[0], data[1]

In [18]:
X.min(), X.max()

(0, 255)

In [19]:
X = X / 255

In [20]:
X.min(), X.max()

(0.0, 1.0)

In [25]:
y_binary = np.where(y >= 0.01, 1, 0)
y_binary.mean()

0.5105893186003683

## Train Test Split

In [10]:
from sklearn.model_selection import train_test_split

In [26]:
x_train, x_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((1737, 299, 299, 3), (435, 299, 299, 3), (1737,), (435,))

## Define VIT

To run the code in a Jupyter notebook, you would first instantiate the `CustomViT` class, then use its methods to create the Vision Transformer (ViT) model, and finally train the model using the provided dataset. Below is a step-by-step guide on how to do this, assuming that the `CustomViT` class object is already defined in your notebook and you have your dataset loaded into `x_train`, `y_train`, `x_test`, and `y_test` variables.

1. **Instantiate the CustomViT Class**

First, create an instance of the `CustomViT` class. You can adjust the parameters based on your dataset and training preferences, or use the defaults as specified in the class definition.

```python
vit_model = CustomViT()
```

2. **Create the ViT Classifier Model**

Next, use the `create_vit_classifier` method to create the model.

```python
model = vit_model.create_vit_classifier()
```

3. **Train the Model**

Now, you can train the model using the `run_experiment` method. This method requires you to pass the model you've just created, along with your training and testing datasets.

```python
history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
```

Make sure you have your dataset (`x_train`, `y_train`, `x_test`, `y_test`) prepared before running this step. The dataset should be pre-processed according to the input requirements of the model, such as resizing images to the expected input shape and normalizing the pixel values.

Here's a complete example in a Jupyter notebook cell:

```python
# Assuming CustomViT class is defined and dataset is loaded and preprocessed
vit_model = CustomViT()

# Create the ViT model
model = vit_model.create_vit_classifier()

# Display the model architecture
model.summary()

# Train and evaluate the model
history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)
```

This code will initialize the custom Vision Transformer model with the specified parameters, compile it, and train it on your dataset. After training, it will also evaluate the model on your test dataset and print the accuracy and top-5 accuracy metrics.

Remember, the variables `x_train`, `y_train`, `x_test`, and `y_test` need to be defined with your dataset. This example assumes that you have already loaded and pre-processed your data to match the input requirements of the ViT model.

In [12]:
! pip install tensorflow-addons

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.8/611.8 kB 9.1 MB/s eta 0:00:00


In [28]:
from utils import CustomViT as myViT

In [29]:
vit_model = myViT()

In [30]:
model = vit_model.create_vit_classifier()

In [31]:
tf.test.gpu_device_name()

'/device:GPU:0'

### Training

In [ ]:
%%time

with tf.device('/device:GPU:0'):
    history = vit_model.run_experiment(model, x_train, y_train, x_test, y_test)

Epoch 1/800
7/7 [==============================] - 21s 1s/step - loss: 21.6098 - accuracy: 0.4843 - top-5-accuracy: 1.0000 - val_loss: 2.4501 - val_accuracy: 0.4368 - val_top-5-accuracy: 1.0000
Epoch 2/800
7/7 [==============================] - 3s 468ms/step - loss: 1.8883 - accuracy: 0.4965 - top-5-accuracy: 1.0000 - val_loss: 1.3140 - val_accuracy: 0.4368 - val_top-5-accuracy: 1.0000
Epoch 3/800
7/7 [==============================] - 3s 468ms/step - loss: 1.5117 - accuracy: 0.4882 - top-5-accuracy: 1.0000 - val_loss: 0.7763 - val_accuracy: 0.4368 - val_top-5-accuracy: 1.0000
Epoch 4/800
7/7 [==============================] - 3s 469ms/step - loss: 1.0709 - accuracy: 0.5048 - top-5-accuracy: 1.0000 - val_loss: 0.7811 - val_accuracy: 0.4368 - val_top-5-accuracy: 1.0000
Epoch 5/800
7/7 [==============================] - 3s 468ms/step - loss: 0.8722 - accuracy: 0.4965 - top-5-accuracy: 1.0000 - val_loss: 0.7178 - val_accuracy: 0.4368 - val_top-5-accuracy: 1.0000
Epoch 6/800
7/7 [=========